The model you are trying to use is gated. Please make sure you have access to it by visiting the model page.To run inference, either set HF_TOKEN in your environment variables/ Secrets or run the following cell to login. 🤗

In [ ]:
from huggingface_hub import login
login(new_session=False)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
!pip install ultralytics
!pip install supervision
!pip install opencv-python
!pip install numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.4/212.4 kB 6.1 MB/s eta 0:00:00


In [ ]:

import cv2
from ultralytics import YOLO
import supervision as sv
from supervision.geometry.core import Point
import csv
import numpy as np


VIDEO_PATH = "mixkit-potholes-in-a-rural-road-25208-hd-ready.mp4"
MODEL_PATH = "Yolov8-fintuned-on-potholes.pt"
OUTPUT_VIDEO = "annotated_output_1.mp4"
OUTPUT_CSV = "pothole_counts_1.csv"

# FRAME_SKIP = 3
ROI_VERTICAL_START = 1/3        # lower 2/3 of frame
POTHOLE_CLASS_ID = 0


model = YOLO(MODEL_PATH)


cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened():
    raise RuntimeError("Error opening video file")

frame_width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)


out = cv2.VideoWriter(
    OUTPUT_VIDEO,
    cv2.VideoWriter_fourcc(*"mp4v"),
    fps / FRAME_SKIP,
    (frame_width, frame_height)
)


csv_file = open(OUTPUT_CSV, "w", newline="")
csv_writer = csv.writer(csv_file)
csv_writer.writerow(["frame_id", "total_potholes", "unique_ids"])


tracker = sv.ByteTrack()


line_y = int(frame_height * (ROI_VERTICAL_START + 0.33))  # middle of ROI
line_start = Point(50, line_y)
line_end   = Point(frame_width - 50, line_y)
counting_line = sv.LineZone(start=line_start, end=line_end)
line_annotator = sv.LineZoneAnnotator()
box_annotator  = sv.BoxAnnotator()

crossed_ids = set()


frame_id = 0

while True:
    success, frame = cap.read()
    if not success:
        break

    # Skip frames for efficiency
    # if frame_id % FRAME_SKIP != 0:
    #     frame_id += 1
    #     continue

    h, w, _ = frame.shape
    roi_offset = int(h * ROI_VERTICAL_START)

    # Crop lower 2/3
    roi_frame = frame[roi_offset:, :]

    # YOLO detection
    results = model(roi_frame, classes=[POTHOLE_CLASS_ID])

    if len(results) == 0:
        frame_id += 1
        continue

    result = results[0]
    detections = sv.Detections.from_ultralytics(result)

    # Shift boxes back to full frame coordinates
    if len(detections) > 0:
        detections.xyxy[:, 1] += roi_offset
        detections.xyxy[:, 3] += roi_offset

    # Assign tracker IDs
    detections = tracker.update_with_detections(detections)

    # Count line crossings safely
    crossed = counting_line.trigger(detections)
    for det_id, did_cross in zip(detections.tracker_id, crossed):
        # Convert arrays to single boolean
        if isinstance(did_cross, (list, tuple, np.ndarray)):
            did_cross = np.any(did_cross)
        if did_cross:
            crossed_ids.add(det_id)

    # Annotate boxes and line
    frame = box_annotator.annotate(frame, detections)
    frame = line_annotator.annotate(frame, counting_line)

    # Overlay total unique potholes count
    total = len(crossed_ids)
    cv2.putText(
        frame,
        f"Total Unique Potholes: {total}",
        (10, 40),
        cv2.FONT_HERSHEY_SIMPLEX,
        1,
        (0, 0, 255),
        2
    )

    # Write frame to video
    out.write(frame)

    # Write CSV: frame ID, total, list of unique tracker IDs
    csv_writer.writerow([frame_id, total, ";".join(map(str, crossed_ids))])

    frame_id += 1

cap.release()
out.release()
csv_file.close()

print("Processing complete!")
print(f"Annotated video saved: {OUTPUT_VIDEO}")
print(f"CSV saved: {OUTPUT_CSV}")





0: 256x640 4 Potholess, 18.7ms
Speed: 1.5ms preprocess, 18.7ms inference, 3.7ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 3 Potholess, 18.0ms
Speed: 1.4ms preprocess, 18.0ms inference, 2.5ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 5 Potholess, 18.0ms
Speed: 1.5ms preprocess, 18.0ms inference, 3.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 3 Potholess, 18.0ms
Speed: 1.4ms preprocess, 18.0ms inference, 2.5ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 4 Potholess, 18.0ms
Speed: 1.5ms preprocess, 18.0ms inference, 2.6ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 5 Potholess, 14.1ms
Speed: 1.5ms preprocess, 14.1ms inference, 3.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 3 Potholess, 14.1ms
Speed: 1.5ms preprocess, 14.1ms inference, 2.9ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 6 Potholess, 14.1ms
Speed: 2.2ms preprocess, 14.1ms inference, 3.7ms postprocess per 